In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import networkx as nx

import cirq
from cirq.contrib.svg import SVGCircuit

In [ ]:
import networkx as nx
SYC23_GRAPH = nx.from_edgelist([
    ((3, 2), (4, 2)), ((4, 1), (5, 1)), ((4, 2), (4, 1)), 
    ((4, 2), (4, 3)), ((4, 2), (5, 2)), ((4, 3), (5, 3)), 
    ((5, 1), (5, 0)), ((5, 1), (5, 2)), ((5, 1), (6, 1)), 
    ((5, 2), (5, 3)), ((5, 2), (6, 2)), ((5, 3), (5, 4)), 
    ((5, 3), (6, 3)), ((5, 4), (6, 4)), ((6, 1), (6, 2)), 
    ((6, 2), (6, 3)), ((6, 2), (7, 2)), ((6, 3), (6, 4)), 
    ((6, 3), (7, 3)), ((6, 4), (6, 5)), ((6, 4), (7, 4)), 
    ((6, 5), (7, 5)), ((7, 2), (7, 3)), ((7, 3), (7, 4)), 
    ((7, 3), (8, 3)), ((7, 4), (7, 5)), ((7, 4), (8, 4)), 
    ((7, 5), (7, 6)), ((7, 5), (8, 5)), ((8, 3), (8, 4)), 
    ((8, 4), (8, 5)), ((8, 4), (9, 4)), 
])

In [ ]:
from recirq.otoc.loschmidt import *
from recirq.quantum_executable import *
from recirq.executable_result import *
from recirq.executable_result import _execute_flattened

In [ ]:
for topo in get_all_diagonal_rect_topologies():
    print(topo, topo.n_qubits)

In [ ]:
np.arange(1, 10, 2)

In [ ]:
%load_ext snakeviz

In [ ]:
#%%snakeviz
exegroup = get_all_diagonal_rect_executables(min_side_length=1, max_side_length=3)
exegroup

In [ ]:
print(exegroup._repr_html_())

```python
def recurse_pg_flatten(pg:Union[Program, ProgramGroup], flat_list, info, parents):
    new_info = dict(info)
    pg_info = dict(pg.info)
    for k in pg_info:
        if k in new_info:
            raise ValueError("Key already exists")
        new_info[k] = pg_info[k]
    new_info=tuple(new_info.items())
    
    if isinstance(pg, ProgramGroup):
        for child in pg.programs:
            recurse_pg_flatten(child, flat_list=flat_list, info=new_info, parents=(pg,) + parents)
    else:
        flat_list.append((pg, new_info, parents))
        
flat_list = []
recurse_pg_flatten(pg, flat_list, {}, ())
for program, info, parents in flat_list:
    print(program, parents)
    
    
import pandas as pd
pd.DataFrame([info for program, info, parents in flat_list])
```

In [ ]:
flat_list = flatten_executable_group(exegroup)
flat_list

In [ ]:
children = _execute_flattened(flat_list)
retree_results(pg, children)

In [ ]:
children[pg]

In [ ]:
assert False

In [ ]:
from recirq.named_topologies import DiagonalRectangleTopology, get_placements
rr = DiagonalRectangleTopology(1, 1).graph
subgs = get_placements(SYC23_GRAPH, rr, plt=plt)

In [ ]:
subg = subgs[0]
sorted(subg.nodes)

In [ ]:
from recirq.otoc.loschmidt import create_diagonal_rectangle_loschmidt_echo_circuit

circuit = create_diagonal_rectangle_loschmidt_echo_circuit(
    qubits = [cirq.GridQubit(r, c) for r, c in sorted(subg.nodes)],
    cycles=4,
    seed=52,
)
SVGCircuit(circuit)

In [ ]:
cirq.final_state_vector(circuit).round(5)

In [ ]:
len(circuit)

In [ ]:
c2 = cirq.Circuit(circuit.all_operations())
print(len(circuit), len(c2))
SVGCircuit(c2)

In [ ]:
import cirq_google as cg
sampler = cg.get_engine_sampler('weber', 'sqrt_iswap')
sampler = cirq.Simulator()

In [ ]:
sorted(circuit.all_qubits())

In [ ]:
def get_success(r):
    return np.all(r.measurements['z'] == np.array([0] * len(circuit.all_qubits())), axis=1).astype(int)

In [ ]:
r1 = sampler.run(circuit + cirq.measure(*sorted(circuit.all_qubits()), key='z'), repetitions=50_000)
get_success(r1)

In [ ]:
r2 = sampler.run(c2      + cirq.measure(*sorted(circuit.all_qubits()), key='z'), repetitions=50_000)
get_success(r2)

In [ ]:
np.mean(get_success(r1))

In [ ]:
np.mean(get_success(r2))